In [ ]:
import math
import numpy as np

In [ ]:
# Assuming data format: {(x, y, z): 'block_type'}
m = {}  # input map
graph = {}
Fsafe = 3
jump_height = 1 

In [ ]:
def add_edge(x1, y1, z1, x2, y2, z2):
    distance = math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2 + (z1 - z2) ** 2)
    graph.setdefault((x1, y1, z1), []).append(((x2, y2, z2), distance))
    
def is_reachable(x, y, z):
    return m.get((x, y, z)) == 'solid' and m.get((x, y, z + 1)) != 'solid'

In [ ]:
def construct_graph():
    for (x, y, z), block_type in m.items():
        if not is_reachable(x, y, z):
            continue
        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]: # we only consider moving 4 dirs
            for dz in range(-Fsafe, jump_height + 1):
                nx, ny, nz = x + dx, y + dy, z + dz
                if nz > z and not is_reachable(x, y, nz - 1):
                    continue
                if nz < z and not is_reachable(nx, ny, nz): # blocked by adjacent block
                    continue
                if is_reachable(nx, ny, nz):
                    add_edge(x, y, z, nx, ny, nz)

In [ ]:
def add_edge_with_resource(x1, y1, z1, k1, x2, y2, z2, k2, ex_cost):
    distance = math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2 + (z1 - z2) ** 2)
    graph.setdefault((x1, y1, z1, k1), []).append(((x2, y2, z2, k2), distance + ex_cost))

def construct_graph_with_resources(K):
    for (x, y, z), block_type in m.items():
        for k in range(K+1):  
            if block_type == 'solid':
                for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                    for dz in [1]: # can only destruct block at z+1
                        nx, ny, nz = x + dx, y + dy, z + dz
                        # no block or not solid or have resource
                        if (nx, ny, nz) not in m or (m[(nx, ny, nz)] != 'solid' or k > 0):
                            nk = k if m.get((nx, ny, nz)) != 'solid' else k-1
                            add_edge_with_resource(x, y, z, k, nx, ny, nz-1, nk, 1) 
            else:
                for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                    for dz in range(-Fsafe, jump_height + 1):
                        nx, ny, nz = x + dx, y + dy, z + dz
                        if (nx, ny, nz) in m and k >= 0:
                            nk = k  # No resource used
                            add_edge_with_resource(x, y, z, k, nx, ny, nz, nk, 1) 
